In [1]:
using LinearAlgebra

"""
    householder_tridiagonal(A)

Transforma una matriz simétrica A en una matriz tridiagonal simétrica T
que es similar a A (conserva los mismos eigenvalores) utilizando el
método de Householder.

# Argumentos
- `A::Matrix{Float64}`: La matriz simétrica n x n a transformar.

# Retorna
- `T::Matrix{Float64}`: La matriz tridiagonal simétrica resultante.
"""
function householder_tridiagonal(matrix::Matrix{Float64})
    # Hacemos una copia para no modificar la matriz original
    A = copy(matrix)
    n = size(A, 1)

    if size(A, 2) != n
        error("La matriz de entrada debe ser cuadrada.")
    end
    if !issymmetric(A)
        error("La matriz de entrada no es simétrica.")
    end

    println("--- Iniciando Método de Householder ---")

    # Paso 1: Bucle principal que itera sobre las columnas
    for k in 1:(n-2)
        # Paso 2: Calcular q
        q = dot(A[(k+1):n, k], A[(k+1):n, k]) # Suma de cuadrados

        # Paso 3: Calcular alpha
        if A[k+1, k] ≈ 0.0
            alpha = -sqrt(q)
        else
            alpha = -sqrt(q) * sign(A[k+1, k])
        end

        # Paso 4: Calcular RSQ
        RSQ = alpha^2 - alpha * A[k+1, k]
        
        # Si RSQ es cero, la columna ya está reducida. Continuamos.
        if RSQ ≈ 0.0
            continue
        end

        # Paso 5: Determinar el vector v
        v = zeros(n)
        v[k+1] = A[k+1, k] - alpha
        v[(k+2):n] = A[(k+2):n, k]

        # Paso 6: Determinar el vector u = (1/RSQ) * A*v
        u = (A * v) / RSQ
        
        # Paso 7: Calcular PROD = v' * u
        # Solo se necesita la parte de v que no es cero.
        PROD = dot(v, u)

        # Paso 8: Determinar el vector z = u - (PROD / 2RSQ) * v
        z = u - (PROD / (2 * RSQ)) * v

        # Paso 9: Actualizar la matriz A -> A - v*z' - z*v'
        # Esta operación matricial reemplaza los Pasos 10, 11 y 12.
        A -= v * z' + z * v'
    end
    
    # Limpieza final: debido a errores de punto flotante, los elementos que
    # deberían ser cero pueden ser números muy pequeños. Los forzamos a ser cero.
    for i in 1:n
        for j in 1:n
            if abs(i-j) > 1
                A[i, j] = 0.0
            end
        end
    end
    
    println("Tridiagonalización completada con éxito.")
    # Paso 15: SALIDA
    return A
end

householder_tridiagonal

In [2]:
# Matriz simétrica de entrada
A_original = [ 4.0  1.0 -2.0  2.0;
               1.0  2.0  0.0  1.0;
              -2.0  0.0  3.0 -2.0;
               2.0  1.0 -2.0 -1.0]

println("Matriz Original:")
display(A_original)

# Aplicamos el algoritmo
T = householder_tridiagonal(A_original)

println("\nMatriz Tridiagonal Resultante (T):")
display(T)


# --- Verificación de Eigenvalores ---
# La propiedad más importante de esta transformación es que conserva los eigenvalores.
eigen_original = eigvals(A_original)
eigen_tridiagonal = eigvals(T)

println("\n--- Verificación de Similitud ---")
println("Eigenvalores de la matriz original A:")
display(sort(eigen_original))
println("\nEigenvalores de la matriz tridiagonal T:")
display(sort(eigen_tridiagonal))

if isapprox(sort(eigen_original), sort(eigen_tridiagonal))
    println("\n✅ Verificación exitosa: Los eigenvalores se conservan.")
else
    println("\n❌ Error en la verificación: Los eigenvalores no coinciden.")
end

Matriz Original:


4×4 Matrix{Float64}:
  4.0  1.0  -2.0   2.0
  1.0  2.0   0.0   1.0
 -2.0  0.0   3.0  -2.0
  2.0  1.0  -2.0  -1.0

--- Iniciando Método de Householder ---
Tridiagonalización completada con éxito.

Matriz Tridiagonal Resultante (T):


4×4 Matrix{Float64}:
  4.0  -3.0       0.0       0.0
 -3.0   3.33333  -1.66667   0.0
  0.0  -1.66667  -1.32      0.906667
  0.0   0.0       0.906667  1.98667


--- Verificación de Similitud ---
Eigenvalores de la matriz original A:


4-element Vector{Float64}:
 -2.1975169774394243
  1.0843644637732177
  2.26853140643124
  6.844621107234968


Eigenvalores de la matriz tridiagonal T:


4-element Vector{Float64}:
 -2.197516977439425
  1.0843644637732168
  2.268531406431242
  6.844621107234966


✅ Verificación exitosa: Los eigenvalores se conservan.
